In [121]:
import pandas as pd
from scipy import stats
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
salesdata = pd.read_csv('/home/prashant/Desktop/competation/bigmartsales.csv')


In [122]:
salesdata.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [123]:
#Converting catagorical values of 'Outlet_Size' to Integer Values
cat_dict ={}
unique_outlet_size_val = salesdata.Outlet_Size.dropna().unique()
copy_outlet_size = salesdata.Outlet_Size.dropna()
count =0
for val in unique_outlet_size_val:
    cat_dict[val] =int(count)
    count =count+1
salesdata['Outlet_Size'] =copy_outlet_size.apply(lambda x :cat_dict[x])


In [124]:
#calculating mode of the column Outlet_Size and mapping each mode to its Outlet_Type
mode_fn =salesdata.pivot_table(columns ='Outlet_Type',values = 'Outlet_Size',aggfunc = (lambda x : (mode(x).mode[0]))).astype(int) 

miss_val = salesdata.Outlet_Size.isnull()
#print(miss_val)
#print(salesdata.loc[miss_val,'Outlet_Type'])
salesdata.loc[miss_val,'Outlet_Size'] = salesdata.loc[miss_val,'Outlet_Type'].apply(lambda x : mode_fn[x])
print('missing values in Outlet_size %s'%salesdata.isnull().sum())

missing values in Outlet_size Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


In [125]:
#Filling missing data for 'Item_Weight' feild
avg_item_weight = salesdata.pivot_table(index = 'Item_Identifier',values ='Item_Weight')
avg_item_weight
miss_weight = salesdata.Item_Weight.isnull()
salesdata.loc[miss_weight , 'Item_Weight'] = salesdata.loc[miss_weight,'Item_Identifier'].apply(lambda x : avg_item_weight[x])
salesdata= salesdata.dropna()
salesdata.isnull().sum()



Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [126]:
salesdata.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,0.0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,0.0,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,0.0,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,2.0,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,1.0,Tier 3,Supermarket Type1,994.7052


In [127]:
#filling Item_fat_content effectively

salesdata.Item_Fat_Content = salesdata.Item_Fat_Content.replace({ 'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})
salesdata.Item_Fat_Content.value_counts()

Low Fat    5516
Regular    3003
Name: Item_Fat_Content, dtype: int64

In [128]:
#FEATURE ENGINEERING ---- creating  a new column
salesdata['total_years'] = 2017 - salesdata['Outlet_Establishment_Year']


In [129]:
#filling 0 with mean item visibility
item_vis = salesdata.pivot_table(index = 'Item_Identifier' , values= 'Item_Visibility')
salesdata.Item_Visibility=salesdata.loc[(salesdata['Item_Visibility'] ==0),'Item_Identifier'].apply(lambda x : item_vis[x])


In [130]:
print(len(salesdata['Item_Type'].unique()))
salesdata['Item_Type'].value_counts()

16


Fruits and Vegetables    1232
Snack Foods              1199
Household                 910
Frozen Foods              855
Dairy                     681
Canned                    649
Baking Goods              647
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [131]:
salesdata.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,total_years
0,FDA15,9.30,Low Fat,NaN,Dairy,249.8092,OUT049,1999,0.0,Tier 1,Supermarket Type1,3735.1380,18
1,DRC01,5.92,Regular,NaN,Soft Drinks,48.2692,OUT018,2009,0.0,Tier 3,Supermarket Type2,443.4228,8
2,FDN15,17.50,Low Fat,NaN,Meat,141.6180,OUT049,1999,0.0,Tier 1,Supermarket Type1,2097.2700,18
3,FDX07,19.20,Regular,0.015274,Fruits and Vegetables,182.0950,OUT010,1998,2.0,Tier 3,Grocery Store,732.3800,19
4,NCD19,8.93,Low Fat,0.008082,Household,53.8614,OUT013,1987,1.0,Tier 3,Supermarket Type1,994.7052,30


In [132]:
rep_food ={'FD':'Consumable_Food','DR':'Drinking_Food','NC':'Non_Consumable'}
salesdata['represent_food'] = salesdata.Item_Identifier.apply(lambda x : x[0:2]).map(lambda x: rep_food[x])
salesdata.represent_food.value_counts()

Consumable_Food    6121
Non_Consumable     1599
Drinking_Food       799
Name: represent_food, dtype: int64

In [133]:
le = LabelEncoder()
salesdata['Outlet'] = le.fit_transform(salesdata['Outlet_Identifier'])
all_variable= list(salesdata.columns.values)
cat_data =[] 
index=0
for i in salesdata.dtypes:
    
    if i == 'object' and all_variable[index] != 'Outlet_Identifier' and all_variable[index] != 'Item_Identifier'   :
        cat_data.append(all_variable[index])
    index = index+1
print(cat_data)
index=0
le = LabelEncoder()
for i in cat_data :
    salesdata[i]= le.fit_transform(salesdata[i])
   

['Item_Fat_Content', 'Item_Type', 'Outlet_Location_Type', 'Outlet_Type', 'represent_food']


In [134]:
salesdata.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,total_years,represent_food,Outlet
0,FDA15,9.30,0,NaN,4,249.8092,OUT049,1999,0.0,0,1,3735.1380,18,0,9
1,DRC01,5.92,1,NaN,14,48.2692,OUT018,2009,0.0,2,2,443.4228,8,1,3
2,FDN15,17.50,0,NaN,10,141.6180,OUT049,1999,0.0,0,1,2097.2700,18,0,9
3,FDX07,19.20,1,0.015274,6,182.0950,OUT010,1998,2.0,2,0,732.3800,19,0,0
4,NCD19,8.93,0,0.008082,9,53.8614,OUT013,1987,1.0,2,1,994.7052,30,2,1


In [135]:
salesdata = pd.get_dummies(salesdata,columns=cat_data)
salesdata.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                  float64
Item_Outlet_Sales            float64
total_years                    int64
Outlet                         int64
Item_Fat_Content_0           float64
Item_Fat_Content_1           float64
Item_Type_0                  float64
Item_Type_1                  float64
Item_Type_2                  float64
Item_Type_3                  float64
Item_Type_4                  float64
Item_Type_5                  float64
Item_Type_6                  float64
Item_Type_7                  float64
Item_Type_8                  float64
Item_Type_9                  float64
Item_Type_10                 float64
Item_Type_11                 float64
Item_Type_12                 float64
Item_Type_13                 float64
Item_Type_14                 float64
I